In [ ]:
import os
# Force CPU backend on Apple Silicon to avoid Metal issues
os.environ['JAX_PLATFORMS'] = 'cpu'

import matplotlib.pyplot as plt
# Disable LaTeX rendering in matplotlib
plt.rcParams["text.usetex"] = False
plt.rcParams["font.family"] = "sans-serif"

from jax import random
from jax import numpy as jnp
from sbijax import plot_loss_profile
import numpy as np
from pathlib import Path
import pandas as pd
import json

In [ ]:
import pickle
from aind_behavior_vrforaging_analysis.sbi_ddm_analysis.simulator import PatchForagingDDM_JAX, create_prior
from aind_behavior_vrforaging_analysis.sbi_ddm_analysis.snle.snle_inference_jax import infer_parameters_snle
from sbijax import NLE
from sbijax.nn import make_maf

# Load saved parameters 
# model_name = 'snle_2M_h128_l8_b256_37feat.pkl'# 'snle_2M_h128_l8_b256_1feat' #''snle_2M_h128_l8_b1024_300feat' # 'snle_2M_h128_l8_b2048_300feat' #'snle_5M_h128_l8_b256_37feat' #'snle_2M_h64_l5_b256_37feat'
# model_name = 'snle_2M_lr0.0001_ts2000_h128_l8_b256_37feat/model'
model_name = 'snle_2M_lr0.0002_ts2000_h128_l8_b256_37feat'
model_name = 'snle_2M_lr0.0005_ts2000_h128_l8_b256_37feat'
model_name = 'snle_2M_lr0.0005_ts2000_h128_l8_b128_37feat' # w negative drift in prior
# model_name = 'snle_2M_lr0.0008_ts2000_h128_l8_b128_37feat' # w negative drift in prior faster learning rate
model_name = 'snle_2M_lr0.0009_ts2000_h128_l8_b128_37feat' #latest model with negative drift in prior and faster learning rate
with open(f'/Users/laura.driscoll/Documents/code/sbi_results/{model_name}/model.pkl', 'rb') as f:
    model_data = pickle.load(f)

# Reconstruct the model
simulator = PatchForagingDDM_JAX(
    max_sites_per_window=100,
    interval_normalization=model_data['config']['interval_normalization']
)
prior_fn = create_prior(
    prior_low=jnp.array(model_data['config']['prior_low']),
    prior_high=jnp.array(model_data['config']['prior_high'])
)

rng_key = random.PRNGKey(model_data['config']['seed']+1)
rng_key, test_key = random.split(rng_key)
test_theta = prior_fn().sample(seed=test_key)
test_x = simulator.simulator_fn(seed=test_key, theta=test_theta)
n_features = test_x.shape[-1]

# Rebuild the flow architecture (must match training)
flow = make_maf(
    n_dimension=n_features,  # obs data dimension
    n_layers=model_data['config']['num_layers'],
    hidden_sizes=(model_data['config']['hidden_dim'], model_data['config']['hidden_dim'])
)

# Create SNLE model
fns = prior_fn, simulator.simulator_fn
snle = NLE(fns, flow)

print("Model reconstructed! Ready for inference.")

In [ ]:
# Disable LaTeX rendering in matplotlib
plt.rcParams["text.usetex"] = False
plt.rcParams["font.family"] = "sans-serif"

_, axes = plt.subplots(figsize=(6, 3))
plot_loss_profile(model_data['losses'], axes)
# axes.set_ylim(bottom=-200,top=0)
plt.show()

In [ ]:
model_name

In [ ]:
# ============================================================================
# Load Real Data Statistics BY ODOR TYPE
# ============================================================================

# Path to processed data
base_path = Path("/Users/laura.driscoll/Documents/data/VR foraging/vr_foraging_data")

# Load batch processing results
results_df = pd.read_csv(base_path / "batch_processing_by_odor_results.csv")
successful_sessions = results_df[results_df['status'] == 'success']

# Define odor types to analyze
odor_types = ['Methyl_Butyrate', 'Alpha_pinene']
odor_display_names = {'Methyl_Butyrate': 'Methyl Butyrate', 'Alpha_pinene': 'Alpha-pinene'}
odor_type = odor_types[0]

In [ ]:
for idx, row in successful_sessions.iterrows():
    # print(idx)
    print(row)

    session_dir = Path(row['session_dir'])
    odor_dir = session_dir / "100_window_data_by_odor" / odor_type

    # Load metadata
    with open(odor_dir / "metadata.json", 'r') as f:
        metadata = json.load(f)

    # Load all windows for this odor
    n_windows = metadata['n_windows']
    session_windows = []
    for i in range(n_windows):
        window = np.load(odor_dir / f"window_{i:03d}.npy")
        session_windows.append(window)

    session_windows = np.array(session_windows)

In [ ]:
param_names = ["drift_rate", "reward_bump", "failure_bump", "noise_std"]
param_labels = [
    "drift_rate: evidence accumulation rate",
    "reward_bump: evidence boost from receiving reward",
    "failure_bump: evidence boost from not receiving reward",
    "noise_std: std of noise in evidence accumulation"
]
cmap = plt.colormaps['rainbow']
number_of_samples = 9

for idx, row in successful_sessions.iterrows():
    # print(idx)
    print(row)

    session_dir = Path(row['session_dir'])
    odor_dir = session_dir / "100_window_data_by_odor" / odor_type

    # Load metadata
    with open(odor_dir / "metadata.json", 'r') as f:
        metadata = json.load(f)

    # Load all windows for this odor
    n_windows = metadata['n_windows']
    session_windows = []
    for i in range(n_windows):
        window = np.load(odor_dir / f"window_{i:03d}.npy")
        session_windows.append(window)

    session_windows = np.array(session_windows)

    # 3. Sample the colormap at regular intervals from 0 to 1
    # This returns an array of RGBA tuples
    gradient = np.linspace(0, 1, number_of_samples)
    colors_rgba = cmap(gradient)
    drift_values = np.linspace(0, 1, number_of_samples)*2

    fig, axes = plt.subplots(1, 4, figsize=(10, 2))
    axes = axes.flatten()

    for session_i in range(n_windows-1):

        n_feat = model_data['config']['n_feat']
        window_data = session_windows[session_i,:,:]
        window_data[:,0] = window_data[:,0]/model_data['config']['interval_normalization']

        if n_feat == 300:
            observed_stats = prepare_raw_data(window_data)
        elif n_feat == 23:
            from aind_behavior_vrforaging_analysis.sbi_ddm_analysis.feature_engineering.enhanced_stats_23 import compute_summary_stats
            observed_stats = compute_summary_stats(window_data)
        elif n_feat == 35:
            from aind_behavior_vrforaging_analysis.sbi_ddm_analysis.feature_engineering.enhanced_stats_35 import compute_summary_stats
            observed_stats = compute_summary_stats(window_data)
        elif n_feat == 37:
            from aind_behavior_vrforaging_analysis.sbi_ddm_analysis.feature_engineering.enhanced_stats_37 import compute_summary_stats
            observed_stats = compute_summary_stats(window_data)

        # --- Run inference ---
        print("\n3. Testing inference...")
        rng_key, subkey = random.split(rng_key)
        posterior_samples, diagnostics = infer_parameters_snle(
        snle,
        model_data['snle_params'],
        observed_stats, 
        model_data['y_mean'], model_data['y_std'],
        num_samples=5_000,
        num_warmup=50,
        num_chains=2,
        rng_key=subkey
        )

        # --- Plot posterior distributions ---
        for i in range(4):

            # Compute histogram
            counts, bins, _ =axes[i].hist(posterior_samples[:, i], bins=30, color=colors_rgba[session_i], edgecolor=None, alpha=0.3)

            # Posterior mode (bin center with max count)
            mode_index = jnp.argmax(counts)
            posterior_mode = (bins[mode_index] + bins[mode_index + 1]) / 2

            # axes[i].axvline(true_theta[i], color=colors_rgba[drift_i], linestyle='-', label="true value",alpha=0.5)
            axes[i].axvline(posterior_mode, color=colors_rgba[session_i], linestyle=':', label='MAP estimate',alpha=0.5)

            axes[i].set_xlabel(param_names[i])
            axes[i].set_ylabel("Frequency")

            axes[i].set_xlim(model_data['config']['prior_low'][i], model_data['config']['prior_high'][i])

        axes[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))

    plt.tight_layout()
    plt.show()


In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(10, 2))
axes = axes.flatten()

# --- Plot posterior distributions ---
for i in range(4):

    # Compute histogram
    counts, bins, _ =axes[i].hist(posterior_samples[:, i], bins=30, color=colors_rgba[session_i], edgecolor=None, alpha=0.3)

    # Posterior mode (bin center with max count)
    mode_index = jnp.argmax(counts)
    posterior_mode = (bins[mode_index] + bins[mode_index + 1]) / 2

    # axes[i].axvline(true_theta[i], color=colors_rgba[drift_i], linestyle='-', label="true value",alpha=0.5)
    axes[i].axvline(posterior_mode, color=colors_rgba[session_i], linestyle=':', label='MAP estimate',alpha=0.5)

    axes[i].set_xlabel(param_names[i])
    axes[i].set_ylabel("Frequency")

    axes[i].set_xlim(model_data['config']['prior_low'][i], model_data['config']['prior_high'][i])

axes[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.tight_layout()
plt.show()

In [ ]:
# Extract data
for session_i in range(2):
    window_data = session_windows[session_i,:,:]

    positions = window_data[ :, 0]
    rewards = window_data[ :, 1]
    stopped = window_data[ :, 2]

    plt.plot(np.where(rewards)[0],window_data[rewards==1,0]/88.58,'ob')
    plt.plot(np.where(rewards==0)[0],window_data[rewards==0,0]/88.58,'xr')
    plt.show()

In [ ]:

for session_i in range(2):

    theta = jnp.array([0.05, 0.5, 0.4, 0.2])  # mid-range parameters
    rng_key, subkey = random.split(rng_key)
    sim_window_data, _ = simulator.simulate_one_window(theta, subkey)

    # Extract data
    positions = sim_window_data[ :, 0]
    rewards = sim_window_data[ :, 1]
    stopped = sim_window_data[ :, 2]

    plt.plot(np.where(rewards)[0],sim_window_data[rewards==1,0],'ob')
    plt.plot(np.where(rewards==0)[0],sim_window_data[rewards==0,0],'xr')
    plt.show()


In [ ]:
vary_theta = 1
param_names = ["drift_rate", "reward_bump", "failure_bump", "noise_std"]
param_labels = [
    "drift_rate: evidence accumulation rate",
    "reward_bump: evidence boost from receiving reward",
    "failure_bump: evidence boost from not receiving reward",
    "noise_std: std of noise in evidence accumulation"
]

number_of_samples = 5
cmap = mpl.colormaps['rainbow']

# 3. Sample the colormap at regular intervals from 0 to 1
# This returns an array of RGBA tuples
gradient = np.linspace(0, 1, number_of_samples)
colors_rgba = cmap(gradient)
theta_values = np.linspace(model_data['config']['prior_low'][vary_theta], 
                           model_data['config']['prior_high'][vary_theta], 
                           number_of_samples)

fig, axes = plt.subplots(1, 4, figsize=(10, 2))
axes = axes.flatten()

# --- Simulate observed data ---
print("\n2. Simulating observed data...")
rng_key = random.PRNGKey(111)
rng_key, subkey = random.split(rng_key)
true_theta = prior_fn().sample(seed=subkey)['theta']

for theta_i, theta in enumerate(theta_values):
    true_theta = true_theta.at[vary_theta].set(theta)
    rng_key, subkey = random.split(rng_key)
    _, observed_stats = simulator.simulate_one_window(true_theta, subkey)
    print(f"   True theta: {true_theta}")

    # --- Run inference ---
    print("\n3. Testing inference...")
    rng_key, subkey = random.split(rng_key)
    posterior_samples, diagnostics = infer_parameters_snle(
    snle,
    model_data['snle_params'],
    observed_stats, 
    model_data['y_mean'], model_data['y_std'],
    num_samples=1000,
    num_warmup=50,
    num_chains=2,
    rng_key=subkey
    )

    # --- Plot posterior distributions ---
    for i in range(4):

        # Compute histogram
        counts, bins, _ =axes[i].hist(posterior_samples[:, i], bins=30, color=colors_rgba[theta_i], edgecolor=None, alpha=0.3)

        # Posterior mode (bin center with max count)
        mode_index = jnp.argmax(counts)
        posterior_mode = (bins[mode_index] + bins[mode_index + 1]) / 2

        axes[i].axvline(true_theta[i], color=colors_rgba[theta_i], linestyle='-', label="true value",alpha=0.5)
        axes[i].axvline(posterior_mode, color=colors_rgba[theta_i], linestyle=':', label='MAP estimate',alpha=0.5)

        axes[i].set_xlabel(param_names[i])
        axes[i].set_ylabel("Frequency")

        axes[i].set_xlim(model_data['config']['prior_low'][i], model_data['config']['prior_high'][i])

    axes[i].legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde

def pairplot(posterior_samples, true_params=None, param_names=None, 
             figsize_per_param=2.5, grid_points=100, fig=None, axes=None,
             color_i = 1):
    """
    Lower-triangle corner plot with:
    - 2D filled KDEs (off-diagonal)
    - 1D KDEs (diagonal)
    - Red 'X' for true parameters
    """

    cmaps = ['Purples', 'Blues', 'Greens', 'Oranges', 'Reds']
    color = plt.cm.get_cmap(cmaps[color_i])(1.)

    if isinstance(posterior_samples, jnp.ndarray):
        posterior_samples = np.array(posterior_samples)
    
    n_params = posterior_samples.shape[1]
    if param_names is None:
        param_names = [f"param{i}" for i in range(n_params)]
    
    if fig is None or axes is None:
        fig, axes = plt.subplots(n_params, n_params, figsize=(figsize_per_param*n_params, figsize_per_param*n_params))
    
    for i in range(n_params):
        for j in range(n_params):
            ax = axes[i, j]
            
            # Only fill lower triangle
            if i < j:
                ax.axis('off')
                continue
            
            # Diagonal: 1D KDE
            if i == j:
                data = posterior_samples[:, i]
                kde = gaussian_kde(data)
                x_grid = np.linspace(data.min(), data.max(), grid_points)
                ax.fill_between(x_grid, kde(x_grid), color=color, alpha=0.5)
                
                if true_params is not None:
                    ax.axvline(true_params[i], color=color, linestyle='-', lw=1)
            
            # Off-diagonal: 2D KDE
            else:
                x = posterior_samples[:, j]
                y = posterior_samples[:, i]
                xy = np.vstack([x, y])
                kde = gaussian_kde(xy)
                x_grid = np.linspace(x.min(), x.max(), grid_points)
                y_grid = np.linspace(y.min(), y.max(), grid_points)
                X, Y = np.meshgrid(x_grid, y_grid)
                Z = kde(np.vstack([X.ravel(), Y.ravel()])).reshape(X.shape)

                # Define the threshold
                threshold = .5

                # Mask values below the threshold
                Z_masked = np.ma.masked_where(Z < threshold, Z)

                ax.contourf(X, Y, Z_masked, levels=20, cmap=cmaps[color_i], alpha=0.7)
                
                if true_params is not None:
                    ax.scatter(true_params[j], true_params[i], c=color, s=50, marker='X', label='True')
            
            # Only label left and bottom axes
            if i < n_params - 1:
                ax.set_xticklabels([])
            else:
                ax.set_xlabel(param_names[j])
            if j > 0:
                ax.set_yticklabels([])
            else:
                ax.set_ylabel(param_names[i])
    
    # Add a legend in the top-left subplot
    handles = []
    if true_params is not None:
        handles.append(plt.Line2D([0], [0], marker='X', color='w', markerfacecolor=color, markersize=8, label='True'))
    axes[0, 1].legend(handles=handles, loc='upper left')
    

In [ ]:
figsize_per_param=2.0
n_params = posterior_samples.shape[1]
fig, axes = plt.subplots(n_params, n_params, figsize=(figsize_per_param*n_params, figsize_per_param*n_params))
pairplot(posterior_samples, true_theta, param_names, figsize_per_param=figsize_per_param, fig=fig, axes=axes)

plt.tight_layout()
plt.show()

In [ ]:
vary_theta = 0
number_of_samples = 5
param_names = ["drift_rate", "reward_bump", "failure_bump", "noise_std"]
param_labels = [
    "drift_rate: evidence accumulation rate",
    "reward_bump: evidence boost from receiving reward",
    "failure_bump: evidence boost from not receiving reward",
    "noise_std: std of noise in evidence accumulation"
]

cmap = mpl.colormaps['rainbow']
# 3. Sample the colormap at regular intervals from 0 to 1
# This returns an array of RGBA tuples
gradient = np.linspace(0, 1, number_of_samples+2)[1:-1]
colors_rgba = cmap(gradient)
theta_values = np.linspace(model_data['config']['prior_low'][vary_theta], 
                           model_data['config']['prior_high'][vary_theta], 
                           number_of_samples+2)[1:-1]

figsize_per_param=2.0
n_params = posterior_samples.shape[1]
fig, axes = plt.subplots(n_params, n_params, figsize=(figsize_per_param*n_params, figsize_per_param*n_params))

# --- Simulate observed data ---
print("\n2. Simulating observed data...")
rng_key = random.PRNGKey(666)
rng_key, subkey = random.split(rng_key)
true_theta = prior_fn().sample(seed=subkey)['theta']

for theta_i, theta in enumerate(theta_values):
    true_theta = true_theta.at[vary_theta].set(theta)
    rng_key, subkey = random.split(rng_key)
    _, observed_stats = simulator.simulate_one_window(true_theta, subkey)
    print(f"   True theta: {true_theta}")

    # --- Run inference ---
    print("\n3. Testing inference...")
    rng_key, subkey = random.split(rng_key)
    posterior_samples, diagnostics = infer_parameters_snle(
    snle,
    model_data['snle_params'],
    observed_stats, 
    model_data['y_mean'], model_data['y_std'],
    num_samples=1000,
    num_warmup=50,
    num_chains=2,
    rng_key=subkey
    )

    # --- Plot posterior distributions ---
    pairplot(posterior_samples, true_theta, param_names, 
             figsize_per_param=figsize_per_param, fig=fig, axes=axes, 
             color_i=theta_i)


plt.tight_layout()
plt.show()

In [ ]:
model_file